## TP 3 Neo4j

## Question1 
pour les premiere question j'ai utiliser la basse de donné moovie ou on a des personnes des films et des liens pour savoir qui a jouer dans qu'elle film qui a ecrit le film ... 

In [39]:
from neo4j import GraphDatabase

uri             = "bolt://localhost:7687"
userName        = "neo4j"
password        = "test1234"
graphDB_Driver  = GraphDatabase.driver(uri, auth=(userName, password))

## Requêtes

In [40]:
#Récupérer l’ensemble des Node d’un graphe
cqlNode= "MATCH (n) RETURN n "
#Récupérer un Node particulier
cqlpart="MATCH (u:Person {name: 'Emil Eifrem'})RETURN u"

#Récupérer l’ensemble des Relationship (ou Path) d’un graphe
cqlPath="MATCH p=()-->() RETURN p"
# Récupérer un Relationship (ou Path) spécifique
cqlSpecPath= "MATCH (m:Movie {title:'RescueDawn'})<-[r:DIRECTED]-(p:Person)RETURN p.name"
# Créer un Node (nous considérons que le graphe existe déjà sur Neo4j) ici on cree la personne Jo
cqlCreateNode="CREATE (n:Person {name: 'John'})"

# Créer un Relationship (en créant les Node correspondant ou non) ici on reprend Jo et on dit qu'il a joué dans Speed Racer
cqlAddRelation="MATCH (person:Person {name: 'John'}), (movie:Movie {title: 'Speed Racer'})CREATE (person)-[:ACTED_IN]->(movie)"




## Execution des requêtes

In [41]:
with graphDB_Driver.session() as graphDB_Session:
    
    nodes = graphDB_Session.run(cqlNode)
    part = graphDB_Session.run(cqlpart)
    paths=graphDB_Session.run(cqlPath)
    spepaths=graphDB_Session.run(cqlSpecPath)
    
    graphDB_Session.run(cqlCreateNode)
    graphDB_Session.run(cqlAddRelation)
   

    
    print("tous les noeuds")
    for node in nodes:

        print(node)
    print("noeud part")
    print(part)
    print('tous les path')
    for path in paths:
        print(path)
    print("path choisi")
    for spepath in spepaths:
        print(spepath)
    
    

tous les noeuds
<Record n=<Node element_id='4:5340822a-2a3c-4306-9e9d-8de64eb14f9b:0' labels=frozenset({'Movie'}) properties={'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}>>
<Record n=<Node element_id='4:5340822a-2a3c-4306-9e9d-8de64eb14f9b:1' labels=frozenset({'Person'}) properties={'born': 1964, 'name': 'Keanu Reeves'}>>
<Record n=<Node element_id='4:5340822a-2a3c-4306-9e9d-8de64eb14f9b:2' labels=frozenset({'Person'}) properties={'born': 1967, 'name': 'Carrie-Anne Moss'}>>
<Record n=<Node element_id='4:5340822a-2a3c-4306-9e9d-8de64eb14f9b:3' labels=frozenset({'Person'}) properties={'born': 1961, 'name': 'Laurence Fishburne'}>>
<Record n=<Node element_id='4:5340822a-2a3c-4306-9e9d-8de64eb14f9b:4' labels=frozenset({'Person'}) properties={'born': 1960, 'name': 'Hugo Weaving'}>>
<Record n=<Node element_id='4:5340822a-2a3c-4306-9e9d-8de64eb14f9b:5' labels=frozenset({'Person'}) properties={'born': 1967, 'name': 'Andy Wachowski'}>>
<Record n=<Node element_

## question 2 


In [8]:
import csv
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "test1234"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [9]:
def create_data(tx, code_commune, nom_commune, code_postal, nom_departement, nom_region):
    tx.run("MERGE (c:Commune {code_commune: $code_commune, nom_commune: $nom_commune, code_postal: $code_postal})", code_commune=code_commune, nom_commune=nom_commune, code_postal=code_postal)
    tx.run("MERGE (d:Departement {nom_departement: $nom_departement})", nom_departement=nom_departement)
    tx.run("MERGE (r:Region {nom_region: $nom_region})", nom_region=nom_region)
    tx.run("""
        MATCH (c:Commune {code_commune: $code_commune})
        MATCH (d:Departement {nom_departement: $nom_departement})
        MATCH (r:Region {nom_region: $nom_region})
        MERGE (c)-[:IN_DEPARTEMENT]->(d)
        MERGE (d)-[:IN_REGION]->(r)
    """, code_commune=code_commune, nom_departement=nom_departement, nom_region=nom_region)

In [10]:
with driver.session() as session:
    with open('communes-departement-region (1).csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            print(row)
            code_commune = row['code_commune_INSEE']
            nom_commune = row['nom_commune']
            code_postal = row['code_postal']
            nom_departement = row['nom_departement']
            nom_region = row['nom_region']
            session.write_transaction(create_data, code_commune, nom_commune, code_postal, nom_departement, nom_region)

C:\Users\Lois\AppData\Local\Temp\ipykernel_3160\1649050360.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_data, code_commune, nom_commune, code_postal, nom_departement, nom_region)


: 